In [1]:
import tensorflow as tf
import numpy as np 
from tensorflow import keras


2025-04-18 23:12:30.734256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745010750.746945   37430 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745010750.750662   37430 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-18 23:12:30.765134: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.datasets import mnist

In [3]:
(train_images,train_labels),(test_images,test_labels) = mnist.load_data()

In [4]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)
encoded_train_label = keras.utils.to_categorical(train_labels, num_classes=10)
encoded_test_label = keras.utils.to_categorical(test_labels, num_classes=10)
print(encoded_train_label.shape)
print(encoded_test_label.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
(60000, 784)
(60000,)
(10000, 784)
(10000,)
(60000, 10)
(10000, 10)


# MLP realted stuff

In [5]:
def model_create(input_dim , output_dim , hidden): # hidden here is a array each number represnet the numpber of nurons in each hidden layer
    model = keras.Sequential()
    model.add(keras.layers.Dense(hidden[0], input_shape=(input_dim,),activation='relu')) # input layers with dim 28*28 for our problem

    for nueron in hidden[1:]:
        model.add(keras.layers.Dense(nueron, activation='relu'))

    model.add(keras.layers.Dense(output_dim, activation='softmax'))

    return model
    
    
    

In [6]:
def extract_weight(model):
    weights = []

    for layer in model.layers:
        layer_weights = layer.get_weights()

        weights.extend([w.flatten() for w in layer_weights])
    return np.concatenate(weights)
    

In [7]:
def return_weights_toModel(model,weight_vector):
    #save shapes
    idx=0

    for i, layer in enumerate(model.layers):  # enumerate to make no iterable items iterable reutrn index i here for example
        layer_weight = layer.get_weights()

        new_weights= [] # new list of weights

        for j,w in enumerate(layer_weight): # j index from enumerate i didnt used it because i dont need it 
            
            n_weights = w.size

            new_w = weight_vector[idx:idx + n_weights].reshape(w.shape)

            new_weights.append(new_w)

            idx += n_weights

        layer.set_weights(new_weights)

    return model  # model with updated layers
    

# fitness function

In [8]:
def fitness_function(x_train, y_train, original_model, weight_vector):

    model_copy = keras.models.clone_model(original_model) # model with new wieght to test with 
    model_copy.build(original_model.input_shape) # without build : erorr weight is not defined in next line when i try to set weight

    return_weights_toModel(model_copy, weight_vector)

    y_pred = model_copy.predict(x_train)

    mse = np.mean((y_pred-y_train)**2)

    return mse
    

In [11]:
# test example 
model = model_create(input_dim=28*28 , output_dim=10 , hidden=[100])

weight_vecotor_test = extract_weight(model)
print(weight_vecotor_test)

mse = fitness_function(train_images,encoded_train_label, model, weight_vecotor_test)
print("mse is : ",mse)

[ 0.04486583 -0.00590992 -0.02085369 ...  0.          0.
  0.        ]
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 634us/step
mse is :  0.09169128556269036


# intialzation function

In [12]:
tf.random.set_seed(24)

def intialize_population(model, population_size, min_val=0 , max_val=1.0):

    weights = extract_weight(model) # to get the length to create the new at the same size

    num_weights = len(weights)

    population = tf.random.uniform(
        shape = (population_size, num_weights),
        minval=min_val,
        maxval=max_val,
        dtype = tf.float32
        
    )

    population_np = population.numpy()


    return population_np
    

In [ ]:
# tes